# NNodely Documentation - Inference

Here are shown all the modalities in which you can make inference with the model.

In [ ]:
# uncomment the command below to install the nnodely package
#!pip install nnodely

from nnodely import *
from nnodely.relation import NeuObj

## Simple inference

To make inference, just call the nnodely framework class by passing the dictionary of inputs. The model will execute a forward pass and return the outputs of the network.

It is mandatory to call the inference with at least 1 possible prediction window for each declared inputs. the framework will predict the maximum possible window given the data

In [ ]:
## Model definition
x = Input('x')
F = Input('F')

px = Parameter('px', tw=0.5, values=[[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.],[1.]])
pf = Parameter('pf', sw=1, values=[[1.]])
next_x = Fir(W=px)(x.tw(0.5))+Fir(W=pf)(F.last())

out = Output('next_x', next_x)

model = Modely()
model.addModel('model',[out])
model.neuralizeModel(0.1)

In [ ]:
## Inference
results = model(inputs={'F':[[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11]]})
print(results)

In [ ]:
results = model(inputs={'F':[[5],[4],[9]],'x':[[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12],[13]]})
print(results)

## Inference with a sampled data

Using the option sampled=True the inference window is left to the user to define by passing every sampling input.

In [ ]:
results = model(inputs={'F':[[5],[2]],'x':[[1,2,3,4,5,6,7,8,9,10,11],[12,13,14,15,16,17,18,19,20,21,22]]}, sampled=True)
print(results)

## Closed Loop and Connect during Inference

During inference time you can define closed loop and connect relations between the variables.

In [ ]:
NeuObj.clearNames()
input1 = Input('in1')
W = Parameter('W', sw=3, values=[[1], [2], [3]])
out = Output('out',Fir(W=W)(input1.sw(3)))

model = Modely(visualizer=None, seed=42)
model.addModel('model', [out])
model.neuralizeModel()

result = model({'in1': [1, 2, 3]}, closed_loop={'in1':'out'})
print(result)

In [ ]:
input2 = Input('in2')
K = Parameter('K', sw=3, values=[[1], [2], [3]])
out2 = Output('out2',Fir(W=K)(input2.sw(3)))

model.addModel('model2', [out2])
model.neuralizeModel()

result = model({'in1': [1, 2, 3]}, connect={'out':'in2'})
print(result)

## Recurrent inference with prediction samples

to do multiple recurrent inference with the state variables just decide the number of samples to predict using the attribute 'prediction_samples'.

it can also be 'auto' meaning that the recurrent inference will continue until there are inputs available.

In [ ]:
result = model({'in1': [1, 2, 3, 4, 5]},closed_loop={'in1':'out'}, prediction_samples=3)
print(result)

## Recurrent inference with number of samples

By specifying the number of samples when doing inference with states variables

The number of samples will specify for how many steps the inference must go on. This can be done only when using states variables 

In [ ]:
result = model({'in1': [1, 2, 3, 4, 5]},closed_loop={'in1':'out'}, prediction_samples=3, num_of_samples = 5)